Initialization

In [10]:
import ODM

# Параметры для инициализации
definitions_path = "./models_product.yml"
mongodb_uri = "mongodb://localhost:27017/"
db_name = "db1"

# Инициализация моделей
ODM.initApp(definitions_path=definitions_path, mongodb_uri=mongodb_uri, db_name=db_name)

# Проверяем, какие классы были созданы в модуле ODM
initialized_classes = [
    name for name, obj in ODM.__dict__.items()
    if isinstance(obj, type)  # Проверяем, что это класс
]
print("Initialized classes:", initialized_classes)
# Получаем все классы из модуля ODM
dynamic_classes = {
    name: obj for name, obj in ODM.__dict__.items()
    if isinstance(obj, type)  # Проверяем, что это класс
}

# Добавляем их в текущее глобальное пространство
globals().update(dynamic_classes)

# Или добавляем их в отдельный словарь для работы
my_environment = {}
my_environment.update(dynamic_classes)

# Проверяем результат
print("Added classes to environment:", list(my_environment.keys()))

Initialized classes: ['Nominatim', 'GeocoderTimedOut', 'Any', 'Point', 'Collection', 'Cursor', 'Model', 'ModelCursor', 'Customer', 'Product', 'Purchase', 'Supplier']
Added classes to environment: ['Nominatim', 'GeocoderTimedOut', 'Any', 'Point', 'Collection', 'Cursor', 'Model', 'ModelCursor', 'Customer', 'Product', 'Purchase', 'Supplier']


In [11]:
from pprint import pprint

Q1

In [14]:
# Query 1: List all purchases of a customer

customer_id = "Tony Stark"  
Q1 = [
    {"$match": {"customer": customer_id}}
]



purchases = list(Purchase.aggregate(Q1))
print("List of customer's purchases:")
pprint(purchases)

List of customer's purchases:
[{'_id': ObjectId('678cf3fc7a7750b77cf5ee90'),
  'customer': 'Tony Stark',
  'products': ['Smartphone', 'Tablet', 'Smart Speaker'],
  'purchase_date': '2024-10-01',
  'purchase_price': 300.0}]


Q2


In [25]:
# Query 2: List all suppliers for the product "Air Conditioner"
product_name = "Air Conditioner"

Q2 = [
    {"$match": {"name": product_name}},
    {"$lookup": {
        "from": "Supplier",
        "localField": "suppliers",
        "foreignField": "name",
        "as": "supplier_details"
    }},
    {"$unwind": "$supplier_details"},
    {"$project": {"supplier_details.name": 1, "_id": 0}}
]



suppliers = list(Product.aggregate(Q2))
print("List of suppliers for 'Air Conditioner':")
pprint(suppliers)

List of suppliers for 'Air Conditioner':
[{'supplier_details': {'name': 'Armstrong Ltd'}},
 {'supplier_details': {'name': 'Fitzgerald, Franklin and Acosta'}}]


Q3


In [16]:
# Query 3: List all different products purchased by a customer
customer_name = "Thomas Mills"  

Q3 = [
    {"$match": {"customer": customer_name}},  
    {"$unwind": "$products"},                 
    {"$group": {"_id": "$products"}},         
    {"$project": {"product_name": "$_id", "_id": 0}}  
]



unique_products = list(Purchase.aggregate(Q3))
print(f"List of different products purchased by {customer_name}:")
pprint(unique_products)

List of different products purchased by Thomas Mills:
[{'product_name': 'Tablet'},
 {'product_name': 'Air Conditioner'},
 {'product_name': 'Monitor'},
 {'product_name': 'Smartwatch'}]


Q4


In [17]:
# Query 4: List products sold by "Modas Paqui" whose name contains "short sleeves"
supplier_name = "Modas Paqui"
product_name_keyword = "short sleeves"

Q4 = [
    {"$match": {
        "suppliers": supplier_name,                 
        "name": {"$regex": product_name_keyword, "$options": "i"}  
    }},
    {"$project": {"name": 1, "_id": 0}}  
]



filtered_products = list(Product.aggregate(Q4))
print(f"Products sold by '{supplier_name}' containing '{product_name_keyword}':")
pprint(filtered_products)

Products sold by 'Modas Paqui' containing 'short sleeves':
[]


I dont have such product and supplier, so I will try another query

In [18]:
supplier_name = "Stark Industries"
product_name_keyword = "Iron Man"

Q4 = [
    {"$match": {
        "suppliers": supplier_name,                 
        "name": {"$regex": product_name_keyword, "$options": "i"}  
    }},
    {"$project": {"name": 1, "_id": 0}}  
]


filtered_products = list(Product.aggregate(Q4))

print(f"Products sold by '{supplier_name}' with the name '{product_name_keyword}':")
pprint(filtered_products) 

Products sold by 'Stark Industries' with the name 'Iron Man':
[{'name': 'Iron Man Suit'}]


Q5

In [24]:
# Query 5: Calculate the total weight and volume of products purchased by a customer on a given day
customer_name = "Thomas Mills"  
purchase_date = "2024-05-27"    

Q5 = [
    {"$match": {"customer": customer_name, "purchase_date": purchase_date}},  
    {"$lookup": {
        "from": "Product",
        "localField": "products",
        "foreignField": "name",
        "as": "product_details"
    }},
    {"$unwind": "$product_details"},  
    {"$group": {
        "_id": None,
        "total_weight": {"$sum": "$product_details.weight"},  
        "total_volume": {"$sum": {"$multiply": ["$product_details.dimensions.length", "$product_details.dimensions.width", "$product_details.dimensions.height"]}}
    }},
    {"$project": {"_id": 0, "total_weight": 1, "total_volume": 1}}  
]



result = list(Purchase.aggregate(Q5))
print(f"Total weight and volume of products purchased by {customer_name} on {purchase_date}:")
pprint(result)

Total weight and volume of products purchased by Thomas Mills on 2024-05-27:
[{'total_volume': 130289, 'total_weight': 111.44999999999999}]


Q6

In [27]:
# Calculate the average number of shipments and stores per month.
Q6 = [
    {"$project": {
        "month": {"$substr": ["$purchase_date", 0, 7]},  
        "shipping_address": 1,                          
        "products": 1                                   
    }},
    {"$lookup": {  
        "from": "Product",
        "localField": "products",
        "foreignField": "name",
        "as": "product_details"
    }},
    {"$unwind": "$product_details"},  
    {"$lookup": { 
        "from": "Supplier",
        "localField": "product_details.suppliers",
        "foreignField": "name",
        "as": "supplier_details"
    }},
    {"$unwind": "$supplier_details"},  
    {"$project": {
        "month": 1,
        "shipping_address": 1,
        "warehouse_address": "$supplier_details.warehouse_addresses"  
    }},
    {"$group": {
        "_id": {"month": "$month"},
        "unique_shipments": {"$addToSet": "$shipping_address"},  
        "unique_warehouses": {"$addToSet": "$warehouse_address"}  
    }},
    {"$project": {
        "month": "$_id.month",
        "monthly_shipments": {"$size": "$unique_shipments"},   
        "monthly_warehouses": {"$size": "$unique_warehouses"}, 
        "_id": 0
    }},
    {"$group": {
        "_id": None,  
        "avg_shipments_per_month": {"$avg": "$monthly_shipments"},    
        "avg_warehouses_per_month": {"$avg": "$monthly_warehouses"}   
    }},
    {"$project": {
        "_id": 0,
        "avg_shipments_per_month": 1,
        "avg_warehouses_per_month": 1
    }}
]


result = list(Purchase.aggregate(Q6))
pprint(result)

[{'avg_shipments_per_month': 1.2727272727272727,
  'avg_warehouses_per_month': 3.6363636363636362}]


Q7

In [28]:
# Query 7: List the three suppliers with the highest turnover volume
Q7 = [
    {"$lookup": {
        "from": "Product",
        "localField": "products",
        "foreignField": "name",
        "as": "product_details"
    }},
    {"$unwind": "$product_details"},  
    {"$group": {
        "_id": "$product_details.suppliers",
        "total_billing_volume": {"$sum": "$product_details.price_with_vat"} 
    }},
    {"$sort": {"total_billing_volume": -1}},  
    {"$limit": 3},  
    {"$project": {"supplier_name": "$_id", "total_billing_volume": 1, "_id": 0}}  
]



top_suppliers = list(Purchase.aggregate(Q7))


print("Top 3 suppliers with highest turnover volume:")
pprint(top_suppliers)

Top 3 suppliers with highest turnover volume:
[{'supplier_name': 'Rodgers-Parrish', 'total_billing_volume': 497.6},
 {'supplier_name': 'Fitzgerald, Franklin and Acosta',
  'total_billing_volume': 452.75},
 {'supplier_name': 'Armstrong Ltd', 'total_billing_volume': 447.65}]


Q8

In [29]:
# Query 8: List warehouses near the White House (100km maximum distance)
latitude = 38.897676   # Latitude for the White House
longitude = -77.03653  # Longitude for the White House
max_distance_km = 100  


Supplier.db.create_index([("warehouse_coordinates", "2dsphere")])

Q8 = [
    {"$geoNear": {
        "near": {"type": "Point", "coordinates": [longitude, latitude]},  
        "distanceField": "distance",  
        "maxDistance": max_distance_km * 1000,  
        "spherical": True  
    }},
    {"$project": {"name": 1, "warehouse_addresses": 1, "distance": 1, "_id": 0}},  
    {"$sort": {"distance": 1}}  
]


nearby_warehouses = list(Supplier.aggregate(Q8))


print("Warehouses within 100km of the White House, ordered by distance:")
pprint(nearby_warehouses)

Warehouses within 100km of the White House, ordered by distance:
[{'distance': 3.33290977848722,
  'name': 'Lowe, Young and Russo',
  'warehouse_addresses': '1600 Pennsylvania Ave NW, Washington, DC 20500, USA'},
 {'distance': 1291.5080559220412,
  'name': 'Barr Group',
  'warehouse_addresses': '701 D St NW, Washington, DC 20024, USA'},
 {'distance': 2318.302729960782,
  'name': 'Riggs PLC',
  'warehouse_addresses': '3109 M St NW, Washington, DC 20007, USA'},
 {'distance': 2772.774030464041,
  'name': 'Sanchez-Lang',
  'warehouse_addresses': '1101 4th St SW, Washington, DC 20024, USA'}]


Q9

In [30]:
# Query 9: List purchases with destination within a specified polygon
polygon_coordinates = [
    [2.174428, 41.403505],   # Barcelona
    [-3.703790, 40.416775],  # Madrid
    [-0.127758, 51.507351],  # London
    [2.352222, 48.856613],   # Paris
    [2.174428, 41.403505]    # Closing the polygon 
]

Q9 = [
    {"$match": {
        "shipping_coordinates": {
            "$geoWithin": {
                "$polygon": polygon_coordinates
            }
        }
    }},
    {"$project": {"customer": 1, "shipping_address": 1, "purchase_date": 1, "_id": 0}}  
]



purchases_in_polygon = list(Purchase.aggregate(Q9))

print("Purchases within specified polygon:")
pprint(purchases_in_polygon)

Purchases within specified polygon:
[{'customer': 'Thomas Mills',
  'purchase_date': '2024-05-27',
  'shipping_address': 'Carrer de Mallorca, 401, 08013 Barcelona, Spain'},
 {'customer': 'Belinda Williams',
  'purchase_date': '2024-09-29',
  'shipping_address': 'Calle de Alcalá, 50, 28014 Madrid, Spain'},
 {'customer': 'Beth Smith',
  'purchase_date': '2023-07-25',
  'shipping_address': 'Calle de Alcalá, 50, 28014 Madrid, Spain'},
 {'customer': 'Barbara Brooks',
  'purchase_date': '2023-09-04',
  'shipping_address': 'Carrer de Mallorca, 401, 08013 Barcelona, Spain'}]


Q10

In [32]:
target_date = "2024-05-27"

#Save in a new table the list of purchases that must be sent from a warehouse on a given day
Q10 = [
    # Step 1: Filter purchases by the given date
    {"$match": {"purchase_date": target_date}},

    # Step 2: Join with the product collection
    {"$lookup": {
        "from": "Product",
        "localField": "products",
        "foreignField": "name",
        "as": "product_details"
    }},

    {"$unwind": "$product_details"}, 

    # Step 3: Join with the supplier collection based on suppliers in products
    {"$lookup": {
        "from": "Supplier",
        "localField": "product_details.suppliers",
        "foreignField": "name",
        "as": "supplier_details"
    }},

    {"$unwind": "$supplier_details"},  

    # Step 4: Project the necessary fields
    {"$project": {
        "customer": 1,
        "shipping_address": 1,
        "purchase_date": 1,
        "warehouse_address": "$supplier_details.warehouse_addresses",
        "_id": 0
    }},

    # Step 5: Save the result to a new collection
    {"$out": "warehouse_shipments"}
]



Purchase.aggregate(Q10)


db = Purchase.db.database
warehouse_shipments = list(db["warehouse_shipments"].find({}, {"_id": 0}))  # Exclude _id for clarity


print(f"Purchases to be shipped from warehouses on the specified date {target_date}:")
pprint(warehouse_shipments)

Purchases to be shipped from warehouses on the specified date 2024-05-27:
[{'customer': 'Thomas Mills',
  'purchase_date': '2024-05-27',
  'shipping_address': 'Carrer de Mallorca, 401, 08013 Barcelona, Spain',
  'warehouse_address': 'Carrer de Mallorca, 401, 08013 Barcelona, Spain'},
 {'customer': 'Thomas Mills',
  'purchase_date': '2024-05-27',
  'shipping_address': 'Carrer de Mallorca, 401, 08013 Barcelona, Spain',
  'warehouse_address': 'Avenida de la Constitución, 3, 41004 Sevilla, Spain'},
 {'customer': 'Thomas Mills',
  'purchase_date': '2024-05-27',
  'shipping_address': 'Carrer de Mallorca, 401, 08013 Barcelona, Spain',
  'warehouse_address': 'United Nations Secretariat Building, 405 E 42nd St, '
                       'New York, NY 10017, USA'},
 {'customer': 'Thomas Mills',
  'purchase_date': '2024-05-27',
  'shipping_address': 'Carrer de Mallorca, 401, 08013 Barcelona, Spain',
  'warehouse_address': 'Paseo del Prado, 2, 28014 Madrid, Spain'}]
